## Dramatron

<img src="./dramatron.png" />

Implementation of [Dramatron](https://arxiv.org/pdf/2209.14958.pdf), Deepmind's generative story prompt model, which starts with a logline and progressively builds elements of a story ending with dialogue for each scene.

In [4]:
import os
import promptz
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

llm = promptz.ChatGPT(
    model='gpt-4',
    api_key=os.environ['OPENAI_API_KEY'],
    org_id=os.environ['OPENAI_ORGANIZATION_ID'],
)

ef = OpenAIEmbeddingFunction(
    api_key=os.environ['OPENAI_API_KEY'],
    model_name="text-embedding-ada-002",
)

promptz.init(llm=llm, ef=ef, use_cache=False)

First define the pydantic types used to generate the story.

In [5]:
from typing import List
from pydantic import BaseModel, Field
from promptz import prompt, store, query


class Character(BaseModel):
    name: str = Field(..., unique=True)
    description: str = None


class Location(BaseModel):
    name: str = Field(..., unique=True)
    description: str = None


class SceneBeat(BaseModel):
    location: str
    plot_element: str
    description: str


class Story(BaseModel):
    logline: str
    title: str = None
    outline: List[SceneBeat] = None
    characters: List[Character] = None
    locations: List[Location] = None

Before generating a story lets define some examples to use as few shots in the prompts. We'll use the Star Wars examples defined in the paper.

In [6]:
star_wars = Story(
    title="Star Wars",
    logline='''
    A science - fiction fantasy about a naive but ambitious farm boy from a 
    backwater desert who discovers powers he never knew he had when he teams 
    up with a feisty princess, a mercenary space pilot and an old wizard warrior 
    to lead a ragtag rebellion against the sinister forces of the evil Galactic 
    Empire.
    ''',
    characters=[
        Character(
            name='Luke Skywalker',
            description='''
            Luke Skywalker is the hero. A naive farm boy, he will 
            discover special powers under the guidance of mentor 
            Ben Kenobi.
            ''',
        ),
        Character(
            name='Ben Kenobi',
            description='''
            Ben Kenobi is the mentor figure. A recluse Jedi warrior, 
            he will take Luke Skywalker as apprentice .
            ''',
        ),
        Character(
            name='Dartha Vader',
            description='''
            Darth Vader is the antagonist. As a commander of the 
            evil Galactic Empire, he controls space station The 
            Death Star.
            ''',
        ),
        Character(
            name='Princess Leia',
            description='''
            Princess Leia holds the plans of the Death Star. She is 
            feisty and brave. She will become Luke's friend.
            ''',
        ),
        Character(
            name='Han Solo',
            description='''
            Han Solo is a brash mercenary space pilot of the 
            Millenium Falcon and a friend of Chebacca. He will
            take Luke on his spaceship.
            ''',
        ),
        Character(
            name='Chewbacca',
            description='''
            Chewbacca is a furry and trustful monster. He is a friend 
            of Han Solo and a copilot on the Millemium Falcon.
            ''',
        ),
    ],
    locations=[
        Location(
            name='Farm',
            description='The farm is a desert planet where Luke Skywalker lives',
        ),
    ],
    outline=[
        SceneBeat(
            location='A farm on planet Tatooine',
            plot_element='The Ordinary World',
            description='Luke Skywalker is living a normal and humble life as a farm boy on his home planet.',
        ),
        SceneBeat(
            location='Desert of Tatooine',
            plot_element='Call to Adventure',
            description='''
            Luke is called to his adventure by robot R2-D2 and Ben Kenobi. 
            Luke triggers R2-D2's message from Princess Leia and is intrigued 
            by her message. When R2-D2 escapes to find Ben Kenobi, Luke follows 
            and is later saved by Kenobi, who goes on to tell Luke about his Jedi 
            heritage. Kenobi suggests that he should come with him.
            '''
        ),
        SceneBeat(
            location="Ben Kenobi's farm",
            plot_element='Refusal of the Call',
            description='''
            Luke refuses Kenobi, telling him that he can take Kenobi and the 
            droids as far as Mos Eisley Spaceport - but he can't possibly leave 
            his Aunt and Uncle behind for some space adventure.
            ''',
        ),
        SceneBeat(
            location='A farm on planet Tatooine',
            plot_element='Crossing the First Threshold',
            description='''
            When Luke discovers that the stormtroopers searching for the droids 
            would track them to his farm, he rushes to warn his Aunt and Uncle, 
            only to discover them dead by the hands of the Empire. When Luke 
            returns to Kenobi, he pledges to go with him to Alderaan and learn 
            the ways of the Force like his father before him.
            ''',
        ),
        SceneBeat(
            location='On spaceship The Millenium Falcon',
            plot_element='Tests, Allies, Enemies',
            description='''
            After Luke, Kenobi, and the droids hire Han Solo and Chewbacca to 
            transport them onto Alderaan, Kenobi begins Luke's training in the 
            ways of the Force. Wielding his father's lightsaber, Kenobi 
            challenges Luke. At first, he can't do it. But then Kenobi tells 
            Luke to reach out and trust his feelings. Luke succeeds.
            ''',
        ),
        SceneBeat(
            location='On spaceship The Millenium Falcon',
            plot_element='Approach to the Inmost Cave',
            description='''
            The plan to defeat the Galactic Empire is to bring the Death Star 
            plans to Alderaan so that Princess Leia's father can take them to 
            the Rebellion. However, when they arrive within the system, the 
            planet is destroyed. They come across the Death Star and are pulled 
            in by a tractor beam, now trapped within the Galactic Empire.
            ''',
        ),
        SceneBeat(
            location='On spacestation The Death Star',
            plot_element='Ordeal',
            description='''
            As Kenobi goes off to deactivate the tractor beam so they can escape, 
            Luke, Han, and Chewbacca discover that Princess Leia is being held on 
            the Death Star with them. They rescue her and escape to the Millennium 
            Falcon, hoping that Kenobi has successfully deactivated the tractor 
            beam. Kenobi later sacrifices himself as Luke watches Darth Vader 
            strike him down. Luke must now avenge his fallen mentor and carry on 
            his teachings.
            ''',
        ),
        SceneBeat(
            location='On spacestation The Death Star',
            plot_element='Reward',
            description='''
            Luke has saved the princess and retrieved the Death Star plans. 
            They now have the knowledge to destroy the Galactic Empire's 
            greatest weapon once and for all.
            ''',
        ),
        SceneBeat(
            location='On spaceship The Millenium Falcon',
            plot_element='The Road Back',
            description='''
            Luke, Leia, Han, Chewbacca, and the droids are headed to the hidden 
            Rebellion base with the Death Star plans. They are suddenly pursued 
            by incoming TIE-Fighters, forcing Han and Luke to take action to 
            defend the ship and escape with their lives - and the plans. They 
            race to take the plans to the Rebellion and prepare for battle.
            ''',
        ),
        SceneBeat(
            location='On fighter ship X-Wing',
            plot_element='The Resurrection',
            description='''
            The Rebels - along with Luke as an X-Wing pilot - take on the Death 
            Star. The Rebellion and the Galactic Empire wage war in an epic space 
            battle. Luke is the only X-Wing pilot that was able to get within the 
            trenches of the Death Star. But Darth Vader and his wingmen are in hot 
            pursuit. Just as Darth Vader is about to destroy Luke, Han returns and 
            clears the way for Luke. Luke uses the Force to guide his aiming as he 
            fires upon the sole weak point of the deadly Death Star, destroying it 
            for good.
            ''',
        ),
        SceneBeat(
            location='At the Rebellion base',
            plot_element='The Return',
            description='''
            Luke and Han return to the Rebellion base, triumphant, as they receive 
            medals for the heroic journey. There is peace throughout the galaxy - at 
            least for now.
            ''',
        ),
    ],
) 
    

The first step Dramatron defines in the generation process is writing a title based on a user defined logline. Instead of writing this manually, we can use a prompt to generate the logline from a title.

In [7]:
title = 'Uncut Gems'
logline = prompt(f'write a logline for {title}')

write a logline for Uncut Gems                                         
"A desperate New York City jeweler with gambling addictions must balance
business, family, and dangerous adversaries as he attempts to stay afloat in his
own high-stakes world after acquiring a rare gemstone."


Next, we define a function that generate an alternative title based on the logline and then use it to add a title to a story instance.

In [8]:
def write_title(story: Story) -> str:
    return prompt(
        'Suggest a alternative, original and descriptive title for a known story.',
        story.logline,
        examples=[
            (
                star_wars.logline,
                "The Death Star's Menace"
            ),
            (
                "Residents of San Fernando Valley are under attack by flying saucers from outer space. The aliens are extraterrestrials who seek to stop humanity from creating a doomsday weapon that could destroy the universe and unleash the living dead to stalk humans who wander into the cemetery looking for evidence of the UFOs. The hero Jeff, an airline pilot, will face the aliens.",
                "The Day The Earth Was Saved By Outer Space."
            ),
        ],
        num_examples=2,
    )

story = Story(
    logline=logline,
)

title = write_title(story)
story.title = title
story

Suggest a alternative, original and descriptive title for a known
story.
"A desperate New York City jeweler with gambling addictions must balance
business, family, and dangerous adversaries as he attempts to stay afloat in his
own high-stakes world after acquiring a rare gemstone."
"Diamonds Under Pressure: A Gambler's Game"                            


Story(logline='"A desperate New York City jeweler with gambling addictions must balance business, family, and dangerous adversaries as he attempts to stay afloat in his own high-stakes world after acquiring a rare gemstone."', title='"Diamonds Under Pressure: A Gambler\'s Game"', outline=None, characters=None, locations=None)

Next we create a list of character objects based on the story logline and title. We then store them as embeddings so we can query them in later steps.

In [9]:
def create_characters(story: Story, n=5) -> List[Character]:
    return prompt(
        f'Create {n} characters for a story.',
        input=story.logline,
        output=List[Character],
        examples=[
            (
                star_wars.logline,
                star_wars.characters,
            ),
        ],
    )

characters = create_characters(story)
store(characters)

Create 5 characters for a story.                                       
"A desperate New York City jeweler with gambling addictions must balance
business, family, and dangerous adversaries as he attempts to stay afloat in his
own high-stakes world after acquiring a rare gemstone."
                                                                           
[                                                                               
{"name": "Howard Ratner", "description": "Howard Ratner is the main character. A
desperate New York City jeweler who finds a rare gemstone and hopes it can pay
off his gambling debts."},
{"name": "Julia", "description": "Julia is Howard's girlfriend who works in his
shop. She supports him even in his most dangerous endeavors."},
{"name": "Dinah", "description": "Dinah is Howard's estranged wife who struggles
to keep their family life separate from his business problems."},
{"name": "Demany", "description": "Demany is Howard's business partner, who
brings 

,id,name,type,description
0,99e581ce-9fe4-4324-a17c-c5b2ad202921,Howard Ratner,character,Howard Ratner is the main character. A despera...
1,7f5c1d75-2e0a-4cc6-9a90-435e7ac0e2f9,Julia,character,Julia is Howard's girlfriend who works in his ...
2,ed4bcb50-6d22-40ba-a3e3-f9ea11c0254d,Dinah,character,Dinah is Howard's estranged wife who struggles...
3,380f5e86-1599-46e9-b4bc-bc0567234edd,Demany,character,"Demany is Howard's business partner, who bring..."
4,98a147f3-9117-42f5-849e-e2e83cc2f19d,Kevin Garnett,character,Kevin Garnett is a client who becomes fascinat...


Now we can generate a plot outline from the story logline, title, and characters.

In [10]:
def write_beats(story: Story, n=10) -> List[SceneBeat]:
    return prompt(
        f'''
        Write a sequence of {n} scene beats for a story a hero's journey structure.
        ''',
        input=dict(logline=story.logline, characters=story.characters),
        output=List[SceneBeat],
        examples=[
            (
                dict(logline=star_wars.logline, characters=star_wars.characters),
                star_wars.outline,
            ),
        ],
    )

beats = write_beats(story)
story.outline = beats.objects
beats

                                                                           
Write a sequence of 10 scene beats for a story a hero's journey structure.      
                                                                            
{'logline': '"A desperate New York City jeweler with gambling addictions
must balance business, family, and dangerous adversaries as he attempts to stay
afloat in his own high-stakes world after acquiring a rare gemstone."',
'characters': {}}
[                                                                          
{"location": "Diamond District Jewelry Shop, NYC", "plot_element": "The Ordinary
World", "description": "Howard Ratner, a jeweler and gambling addict, juggles
business and family in New York's Diamond District."},
{"location": "Diamond District Jewelry Shop, NYC", "plot_element": "Call to
Adventure", "description": "Howard acquires a rare gemstone that could make him
a fortune. It's his chance to make his biggest score yet."},
{"location": "Di

,type,location,plot_element,description
0,scenebeat,"Diamond District Jewelry Shop, NYC",The Ordinary World,"Howard Ratner, a jeweler and gambling addict, ..."
1,scenebeat,"Diamond District Jewelry Shop, NYC",Call to Adventure,Howard acquires a rare gemstone that could mak...
2,scenebeat,"Diamond District Jewelry Shop, NYC",Refusal of the Call,As Howard showcases the gem to a potential buy...
3,scenebeat,"Basketball Game, NYC",Crossing the First Threshold,"To convince Garnett of the gem's power, Howard..."
4,scenebeat,"Diamond District, various locations","Tests, Allies, Enemies","Howard juggles rescheduling the auction, retri..."
5,scenebeat,Handling the gem in various locations,Approach to the Inmost Cave,Howard retrieves the gem and pawns it to place...
6,scenebeat,"Sportsbook Office, Philly",Ordeal,"The potential payoff is massive, but when his ..."
7,scenebeat,"Basketball Game, NYC",Reward,Howard's high-stakes bets placed on Garnett's ...
8,scenebeat,"Diamond District Jewelry Shop, NYC",The Road Back,"With the winning in hand, Howard now must conf..."
9,scenebeat,"Diamond District Jewelry Shop, NYC",The Resurrection,"In a shocking twist, an embittered enemy shoot..."


Next we extract the scene 'beats' from the plot outline generated in the previous step. These are the main events that happen in the story.

Each scene beat has a location name so we can use this to extract location objects. These are stored like characters. For some reason the examples in the paper don't use locations from Star Wars. They also use the story logline and just the name of the location from the scene beat instead of the description of the scene. It's unclear why these decisions were made, but for consistency we'll do the same.

In [11]:
def extract_locations(story: Story) -> List[Location]:
    return prompt(
        '''
        Generate a location based on the story logline and location name. 
        ''',
        input=[dict(logline=story.logline, name=beat.location) for beat in story.outline],
        output=Location,
        examples=[
            (
                dict(
                    logline="Morgan adopts a new cat, Misterio, who sets a curse on anyone that pets them.",
                    name="The Adoption Center",
                ),
                Location(
                    name="The Adoption Center",
                    description='''
                    The Adoption Center is a sad place, especially for an unadopted 
                    pet. It is full of walls and walls of cages and cages. Inside of 
                    each is an abandoned animal, longing for a home. The lighting is 
                    dim, gray, buzzing fluorescent.
                    ''',
                )
            ),
            (
                dict(
                    logline="James finds a well in his backyard that is haunted by the ghost of Sam.",
                    name="The Well",
                ),
                Location(
                    name="The Well",
                    description='''
                    The well is buried under grass and hedges. It is at least 
                    twenty feet deep, if not more and it is masoned with stones. 
                    It is 150 years old at least. It stinks of stale, standing 
                    water, and has vines growing up the sides. It is narrow enough 
                    to not be able to fit down if you are a grown adult human.
                    ''',
                )
            ),
            (
                dict(
                    logline="Mr. Dorbenson finds a book at a garage sale that tells the story of his own life. And it ends in a murder! ",
                    name="The Garage Sale",
                ),
                Location(
                    name="The Garage Sale",
                    description='''
                    It is a garage packed with dusty household goods and antiques. 
                    There is a box at the back that says FREE and is full of paper 
                    back books.
                    ''',
                )
            ),
        ],
        num_examples=3,
    )

locations = extract_locations(story)
store(*locations)

                                                                           
Generate a location based on the story logline and location name.               
                                                                            
{'logline': '"A desperate New York City jeweler with gambling addictions
must balance business, family, and dangerous adversaries as he attempts to stay
afloat in his own high-stakes world after acquiring a rare gemstone."', 'name':
'Diamond District Jewelry Shop, NYC'}
{"name": "Diamond District Jewelry Shop, NYC", "description": "\n
The shop is located in New York City's renowned Diamond District. It is\n
filled with valuable gemstones, precious metals and sophisticated
jewelry\ndesigns. It is small and compact, with a haze of panic and desperation
in the air.\n                    There's a dingy back office, littered with
betting slips and overdue\nbills. The air is ripe with the smell of metal, dust
and fear.\n                    "}
                   

,id,name,type,description
0,99e581ce-9fe4-4324-a17c-c5b2ad202921,Howard Ratner,character,Howard Ratner is the main character. A despera...
1,7f5c1d75-2e0a-4cc6-9a90-435e7ac0e2f9,Julia,character,Julia is Howard's girlfriend who works in his ...
2,ed4bcb50-6d22-40ba-a3e3-f9ea11c0254d,Dinah,character,Dinah is Howard's estranged wife who struggles...
3,380f5e86-1599-46e9-b4bc-bc0567234edd,Demany,character,"Demany is Howard's business partner, who bring..."
4,98a147f3-9117-42f5-849e-e2e83cc2f19d,Kevin Garnett,character,Kevin Garnett is a client who becomes fascinat...
5,514eb6f4-791b-49a1-bc6d-631a97877c9d,"Diamond District Jewelry Shop, NYC",NaN,\n The shop is located in N...
6,f34a1bbd-b8e6-4156-beb3-5c0c66636a63,"Diamond District Jewelry Shop, NYC",NaN,\n Tucked into an unassumin...
7,f22dceac-dc02-458a-b652-7f215b799a0e,"Diamond District Jewelry Shop, NYC",NaN,\n Located in the heart of ...
8,2e3ea88f-a5a3-4210-9e4f-45c95a882cc8,"Basketball Game, NYC",NaN,\n The atmosphere in the bu...
9,f6111a86-167d-46ee-951d-b3610308325a,"Diamond District, various locations",NaN,\n The Diamond District is ...


Finally, we can write a script for each scene using the generated scene beats and the characters and locations we generated earlier. Characters and locations are queried using the scene beat data so that the most relevant entries are found using the stored embeddings. Again, the example follows the paper.

In [12]:
def write_script(story: Story, n=None) -> str:
    inputs = [
        dict(
            location=query(beat.location, where={'type': 'location'}).first,
            characters=query(beat.description, where={'type': 'character'}).objects,
            plot_element=story.outline[i].plot_element,
            logline=story.logline,
            previous_beat=story.outline[i-1].description if i > 0 else None,
            beat=story.outline[i].description,
        ) for i, beat in enumerate(story.outline[:n or len(story.outline)])
    ]
    
    return prompt(
        '''
        Write a script based on the characters and locations
        and the events described in the scene beat.
        ''',
        input=inputs,
        examples=[
            (
                dict(
                    location=Location(
                        name='Cockpit of an airplane',
                        description='''
                        Cockpit of a modern passenger airplane, American Flight 812.
                        ''',
                    ),
                    characters=[
                        Character(
                            name='Jeff',
                            description='''
                            Jeff is the hero. A man in his early forties, he tries 
                            to stay calm in all circumstance. Jeff is now a airline 
                            pilot.
                            ''',
                        ),

                        Character(
                            name='Danny',
                            description='''
                            Danny, a young airplane pilot in his thirties, is eager 
                            to learn but can quickly lose his composture. Danny is 
                            enamored of Edith.
                            ''',
                        ),
                        Character(
                            name='Edith',
                            description='''
                            Edith, an experienced stewardess with a good sense of 
                            humour, is trustworthy and dependable. Edith likes to 
                            tease Danny.
                            ''',
                        ),
                    ],
                    plot_element=" Crossing the First Threshold",
                    logline="Residents of San Fernando Valley are under attack by flying saucers from outer space. The aliens are extraterrestrials who seek to stop humanity from creating a doomsday weapon that could destroy the universe and unleash the living dead to stalk humans who wander into the cemetery looking for evidence of the UFOs. The hero Jeff, an airline pilot, will face the aliens.",
                    previous_beat='''
                    Flight captain Jeff reluctantly leaves his wife Paula to go 
                    for a two-day flight.
                    ''',
                    beat='''
                    At the cockpit, flight captain Jeff is preoccupied by the 
                    flying saucer appearances and graveyard incidents in his home 
                    town, where he left wis wife Paula. Without success, co-pilot 
                    Danny and stewardess Edith try to reassure him.
                    ''',
                ),
                '''
                DANNY
                You're mighty silent this trip, Jeff.
                JEFF
                Huh?
                DANNY
                You haven't spoken ten words since takeoff.
                JEFF
                I guess I'm preoccupied, Danny.
                DANNY
                We've got thirty-three passengers back there that have time to 
                be preoccupied. Flying this flybird doesn't give you that 
                opportunity.
                JEFF
                I guess you're right, Danny.
                DANNY
                Paula?
                JEFF
                Yeah.
                DANNY
                There's nothing wrong between you two?
                JEFF
                Oh no, nothing like that. Just that I'm worried, she being there 
                alone and those strange things flying over the house and those 
                incidents in the graveyard the past few days. It's just got me 
                worried.
                DANNY
                Well, I haven't figured out those crazy skybirds yet but I give 
                you fifty to one odds the police have figured out that cemetery 
                thing by now.
                ( Enter EDITH )
                JEFF
                I hope so.
                EDITH
                If you're really that worried Jeff why don't you radio in and 
                find out? Mac should be on duty at the field by now. He could call 
                Paula and relay the message to you.
                DANNY
                Hi Edith.
                EDITH
                Hi Silents. I haven't heard a word from this end of the plane since 
                we left the field.
                DANNY
                Jeff's been giving me and himself a study in silence.
                EDITH
                You boys are feudin'?
                JEFF
                Oh no Edie, nothing like that.
                DANNY
                Hey Edie, how about you and me balling it up in Albuquerque?
                EDITH
                Albuquerque? Have you read that flight schedule Boy?
                DANNY
                What about it?
                EDITH
                We land in Albuquerque at 4 am. That's strictly a nine o'clock 
                town.
                DANNY
                Well I know a friend that ll help us --
                EDITH
                Let's have a problem first, huh Danny.
                DANNY
                Ah he's worried about Paula.
                EDITH
                I read about that cemetery business. I tried to get you kids to 
                not buy too near one of those things. We get there soon enough as 
                it is.
                DANNY
                He thought it'd be quiet and peaceful there.
                EDITH
                No doubt about that. It's quiet alright, like a tomb. I'm sorry 
                Jeff, that was a bad joke.
                '''
            )
        ]
    )


script = []
for scene in write_script(story):
    script.append(scene)

                                                                           
Write a script based on the characters and locations                            
and the events described in the scene beat.                                     
                                                                            
{'location': {'name': 'Diamond District Jewelry Shop, NYC', 'id':
'f22dceac-dc02-458a-b652-7f215b799a0e', 'type': nan, 'description': '\n
Located in the heart of New York City, the Diamond District Jewelry Shop is a
bustling hive of activity. It is filled with rows and rows of sparkling jewels
encased in glass displays, the light refracted and scattered in thousands of
different directions. The constant chime of the door rings out as customers come
and go, their conversations drowned out by the low hum of traffic just outside.
Despite the bright sparkle of its wares, the shop holds a tense atmosphere, the
stakes of every transaction high and palpable.\n                    '},


That covers the basic implementation described in the paper, but we can extend this by breaking the generated scripts into storyboards and then illustrating those frames with an image model.

In [14]:
script

['"INT. DIAMOND DISTRICT JEWELRY SHOP - DAY\n\nHOWARD RATNER, mid-40s, nervous energy personified, is behind the counter dealing with several CUSTOMERS. KEVIN GARNETT, DEMANY, JULIA, and DINAH are amongst the hustle and bustle. Chime bells ring signaling the door opening frequently. \n\nHOWARD\n(Double dealing with two customers)\nOf course, sir, these diamonds are of highest quality, as rare as they come. And madam, I assure you this necklace will catch every eye in the room.\n\nJust then, Howard\'s cell phone rings. He pulls it out, checking the caller ID. It\'s a bookie. \n\nHOWARD\n(To the customers)\nJust a minute.\n\nCuts the call, puts it back in pocket.\n\nHOWARD\n(Back to customers)\nSorry about that. \n\nHe turns to JULIA who\'s nearby.\n\nHOWARD\n(whispering)\nJulia, check out the opals for Mr. Garnett.\n\nJulia nods and heads in the direction of Garnett.\n\nHOWARD\n(to DINAH)\nAnd Dinah, can you handle the cash register?\n\nDinah nods, less than thrilled with the task.\n\nD

In [15]:
import os

def write_to_files(directory, lines):
    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)

    # Iterate over the lines
    for i, line in enumerate(lines, 1):
        # Remove the surrounding quotes and split into lines
        stripped_line = line.strip('"').split('\\n')

        # Open the output file
        with open(f'{directory}/scene{i}.txt', 'w') as file:
            # Write the lines to the file
            file.write('\n'.join(stripped_line))

# Example usage:
write_to_files('uncut gems', script)

In [16]:
def storyboard(script, beat, logline):
    return prompt(
        '''
        Break the script down into 6-12 frames and describe each frame
        so that it can be illustrated. Use any info about the story,
        characters, locations, etc. that you need to describe the frames. 
        ''',
        dict(
            logline=logline,
            location=query(beat.location, where={'type': 'location'}).first,
            characters=query(beat.description, where={'type': 'character'}).objects,
            script=script,
        ),
        List[str],
    )

frames = storyboard(script[0], story.outline[0], story.logline)
frames

                                                                           
Break the script down into 6-12 frames and describe each frame                  
so that it can be illustrated. Use any info about the story,                    
characters, locations, etc. that you need to describe the frames.               
                                                                            
{'logline': '"A desperate New York City jeweler with gambling addictions
must balance business, family, and dangerous adversaries as he attempts to stay
afloat in his own high-stakes world after acquiring a rare gemstone."',
'location': {'name': 'Diamond District Jewelry Shop, NYC', 'id': 'f22dceac-
dc02-458a-b652-7f215b799a0e', 'type': nan, 'description': '\n
Located in the heart of New York City, the Diamond District Jewelry Shop is a
bustling hive of activity. It is filled with rows and rows of sparkling jewels
encased in glass displays, the light refracted and scattered in thousands of
differe

,type,output
0,str,"""Frame 1: A wide shot of the busy Diamond Dist..."
1,str,"""Frame 2: A close shot of Howard Ratner, mid-4..."
2,str,"""Frame 3: Medium shot capturing Kevin Garnett,..."
3,str,"""Frame 4: Close-up of Howard's cell phone as i..."
4,str,"""Frame 5: Shot focusing on Howard reassuring h..."
5,str,"""Frame 6: Mid-shot of Howard whispering to Jul..."
6,str,"""Frame 7: Close-up of Julia nodding in underst..."
7,str,"""Frame 8: Medium shot of Howard instructing Di..."
8,str,"""Frame 9: Close-up of Dinah's less than thrill..."
9,str,"""Frame 10: Wide shot of Dinah moving towards t..."


In [17]:
from promptz import DALL_E

dall_e = DALL_E(
    api_key=os.environ['OPENAI_API_KEY'],
    org_id=os.environ['OPENAI_ORGANIZATION_ID'],
)

def draw_frames(frames):
    return prompt(
        '''
        Illustrate the action and scene described in the frame. 
        ''',
        frames,
        llm=dall_e,
        silent=True,
    )

descriptions = frames['output'].to_list()
images = draw_frames(descriptions)

def display_image(image, description):
    import base64
    from IPython.display import display, Image
    image_bytes = base64.b64decode(image)
    print(description)
    display(Image(data=image_bytes))

In [18]:
display_image(next(images), next(descriptions))

: 

: 